In [124]:
import numpy as np
import numpy.linalg as la

A = np.array([[1, 1, 1],
              [3, 1, -3],
              [1, -2, -5]], dtype = float)

x0 = np.array([0, 1, 1], dtype = float)
x1 = np.array([3, -2, 1], dtype = float)
x2 = np.array([-2, -6, 1], dtype = float)

In [125]:
def LU_dcomp(matrix):
    
    n = matrix.shape[0]
    P = np.eye(n, dtype = float) 
    U = matrix.copy()
    L = np.eye(n, dtype = float)

    for i in range(n):
        row_ind = np.argmax(U[i:, i]) + i   
        if row_ind != i:
            U[[i, row_ind], :] = U[[row_ind, i], :]
            P[[i, row_ind], :] = P[[row_ind, i], :]
            L[[i, row_ind], :i] = L[[row_ind, i], :i]
            
        for j in range(i+1, n):
            z = U[j, i] / U[i, i]
            L[j, i] = z
            
            for k in range(i, n):
                U[j, k] = U[j, k] - z * U[i, k]
    return L,U,P

def back_sub(A, b):
    n = len(b)
    x = np.zeros(len(b), dtype = float)
    for i in range(n - 1, -1, -1):
        if A[i, i] == 0:
            raise ValueError("Matrix is singular")
        x[i] = b[i]
        for j in range(i+1, n):
            x[i] -= A[i, j] * x[j]
        x[i] = x[i] / A[i, i]
    return x

def fwd_elim(A, b):
    n = len(b)
    x = np.zeros(len(b), dtype = float)
    
    for i in range(n):
        if A[i, i] == 0:
            raise ValueError("Matrix is singular")
        x[i] = b[i]
        for j in range(0, i):
            x[i] -= A[i, j] * x[j]
        x[i] = x[i] / A[i, i]
    
    return x

In [126]:
def RQI(A, x0, tol, max_iter):
    n = A.shape[0]
    I = np.eye(n)
    x = x0 / la.norm(x0)
    sigma = x.T @ A @ x
    
    for k in range(max_iter):
        L, U, P = LU_dcomp(A - sigma * I)
        z = fwd_elim(L, P @ x)
        y = back_sub(U, z)
        x_new = y / la.norm(y)
        
        sigma_new = x_new.T @ A @ x_new

        # Checking for convergence
        if (la.norm(x_new - x) < tol):
            count = k
            break
        x = x_new
        sigma = sigma_new
    
    return sigma, x, count

In [127]:
evalue, evector, count = RQI(A, x0, 1.0e-10, 100)
print(evalue)
print(evector)
print(count)
print(A @ evector - evalue * evector)

-6.213349418004875
[ 0.18297672 -0.44143747 -0.87843752]
4
[ 5.55111512e-15  7.10542736e-15 -2.66453526e-15]


In [128]:
evalue, evector, count = RQI(A, x1, 1.0e-10, 100)
print(evalue)
print(evector)
print(count)
print(A @ evector - evalue * evector)

0.335556742666732
[ 0.54682533 -0.74512799  0.38179358]
4
[1.14741550e-13 8.78186412e-14 6.77236045e-15]


In [129]:
evalue, evector, count = RQI(A, x2, 1.0e-10, 100)
print(evalue)
print(evector)
print(count)
print(A @ evector - evalue * evector)

2.8777926753380503
[ 0.38557643  0.90479582 -0.18076322]
4
[0. 0. 0.]
